# Análise de Sentimento à partir do Twitter em Português

Por ser uma rede com restrição a 280 caracteres, os usuários são obrigados a se expressar com objetividade, com menor margem para figuras de linguagem complexas.
Foi concebido para o envio de informações efêmeras, no início o objetivo era apenas publicar o que os usuários estavam fazendo naquela hora. 

Hoje é um verdadeiro "radar" dos acontecimentos mundiais. É possível ter uma boa ideia do que se passa no mundo, através do Twitter, o que faz dele uma boa fonte de dados para criação do modelo.

Criado o modelo, o objetvo é utilizá-lo para "medir" o sentimento de mensagens com determinado conteúdo ou hashtag. Dada quantidade de mensagens publicadas, é impossível lê-las todas para descobrir se o sentimento geral sobre determinado assunto é bom ou ruim. Um modelo pode fazer isso em minutos.


Iniciamos com a biblioteca básica

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

Para extração dos tuítes, utilizaremos o Tweepy

In [ ]:
import tweepy
from tweepy import OAuthHandler
import json
import pandas as pd
from tweepy.streaming import StreamListener
from tweepy import Stream
from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

#tratamento das Strings
import re  
import string



Para acessar o Twitter e "puxar" o conteúdo, você vai precisar se cadastrar  e obter as chaves de acesso da API do Twitter.
Você consegue isso criando e cadastrando uma aplicação no próprio twitter


In [ ]:
#Chaves de acesso

consumer_key   = 'nbnvbcnvbncbvncbvncbvncbvn'
consumer_secret = 'nbnvbcnvbncbvncbvncbvncbvn'
access_token = 'nbnvbcnvbncbvncbvncbvncbvn'
access_secret = 'nbnvbcnvbncbvncbvncbvncbvn'

# Captura

Função para extrair as mensagens

In [ ]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

def tweetSearch(query, limit = 10000, language = "pt"):
    message,favorite_count,followers_count, retweet_count,created_at,user_name,retificado=[],[],[],[],[],[],[]
    shortword = re.compile(r'\W*\b\w{1,2}\b')  #elimina palavras curtas menos de 2 letras
    for tweet in tweepy.Cursor(api.search,q=query , count=limit, result_type="recent", include_entities=True,lang=language, tweet_mode='extended').items(limit):
      if (not tweet.retweeted) and ('RT @' not in tweet.full_text): #exclui retuítes
        text=tweet.full_text
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"_", "", text)
        text = re.sub(r"@\S+", "", text)   #tira referências a usuários
        text = re.sub(r"#\S+", "", text)   #tira hashtags
        text = re.sub(r"\n", "", text)   #tira new line
        message.append(text)
        encurtado=shortword.sub('',text)  # tira palavras pequenas
        retificado.append(encurtado.lower())
        created_at.append(tweet.created_at)
        user_name.append(tweet.user.name)
        retweet_count.append(tweet.retweet_count)
        followers_count.append(tweet.user.followers_count)
        df=pd.DataFrame({'Message':message, 
                'Retificado':retificado,
                'Criado':created_at,
                'Usuário':user_name,
                'Retweet Count':retweet_count,
                'Seguidores':followers_count})
    #df.to_csv("Search Tweets.csv")
    df['Retificado'] = df['Retificado'].str.replace("[^a-zA-Z\xC0-\xFF]+", " ")  #tira vírgula e etc
    return df

A função acima retorna um dataframe com as mensagens e as informações. Cria ainda a coluna "Retificado", com o texto limpo de quaisquer caracteres que não sejam letras e palavras com 2 letras ou menos. elimina também mensagens retuitadas

É possível captar mais informações. Basta consultar a referência no próprio Twitter

Uso da função:

In [ ]:
positivos=tweetSearch("#positivos", limit=10)  # retorna um Dataframe com até 10 tuites
positivos

O mesmo para capturar tuítes "negativos". 

In [ ]:
negativos=tweetSearch("#negativos", limit=10)  # retorna um Dataframe com até 10 tuites

A escolha das hashtags que retornarão tuítes negativos e positivos é pessoal. Uma maneira comum é a busca por emoticons :)  e :(.  Acredito que a busca por #lixo ou #vergonha retornem mensagens negativas. Por outro lado #ferias #amor estejam presentes em mensagens positivas. Um modelo mais eficiente, depende de escolhas acertadas de conteúdo ou hashtag.

Abaixo, Eliminamos mensagens replicadas e adicionamos a coluna de "Tag"

In [ ]:
negativos.drop_duplicates(subset='Message',keep='first', inplace=True)  #elimina duplicados mantém o primeiro 
positivos.drop_duplicates(subset='Message',keep='first', inplace=True)  

negativos['Tag']=0
positivos['Tag']=1

Podemos querer capturar um número grande de mensagens, porém o twitter tem limitações. Uma dica é criar um banco SQL para salvar os tuites obtidos

In [ ]:
import sqlite3
conn = sqlite3.connect("tuites.db") # ou use :memory: para botá-lo na memória RAM
cursor = conn.cursor()

In [ ]:
positivos.to_sql(name='Positivos',con=conn,  if_exists='append')
negativos.to_sql(name='Negativos',con=conn,  if_exists='append')

Visualize as tabelas contidas na base de dados tuites.db

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# Stemming

Com o código acima, é possível obter dois dataframes (positivo e negativo) com milhares de mensagens. Recomendo no mínimo 20 mil de cada. As mensagens estão na coluna "Retificado"

O passo seguinte é o "stemming", que consiste em reduzir as palavras ao seu radical. Por exemplo: amigo, amiga, amigável, fica reduzido a amig , economizando posições em nosso dicionário. 
Isso reduz o tamanho vocabulário sem comprometer a informação,assim podemos obter um modelo mais eficiente pela relação features / instâncias de treinamento.

Já existe um pacote pronto que realiza isso em PORTUGUÊS, o snowball Stemmer.  


In [ ]:
# primeiro juntamos os dois dataframes em um único
data=[positivos,negativos]
dataset=pd.concat(data)
dataset=dataset.sample(frac=1).reset_index(drop=True)

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("portuguese")

def stem_sentences(sentence):
    tokens = sentence.split()
   # tokens = [x for x in tokens if not x in stop_words]
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

  



In [ ]:
dataset['Retificado']=dataset['Retificado'].apply(stem_sentences)

#dê uma olhada como fica
dataset['Retificado']  

# Vetorizando

Agora que já reduzimos as palavras, o passo seguinte é a vetorização. Nesse processo, todas as palavras são colocadas em ordem alfabética e nosso dataset será codificado em uma matriz:  TAMANHO DO DATASET X TAMANHO DO VOCABULÁRIO
Lembrando que o vocabulário é formado pelas palavras de todas as mensagens "stemmizadas".

Para a vetorização, vamos utilizar uma biblioteca do scikitlearn, que:
- lista todas as palavras (previamente "stemmizadas") diferentes. Umas 18 mil
- as coloca em ordem alfabética
- vetoriza

Assim, cada mensagem não importa o tamanho, se tornará um vetor de 18 mil palavras(+/-) onde 1 é a presença da palavra e zero, ausência.

Todas as mensagens que no futuro serão analizadas pelo modelo, deverão ser vetorizadas com o mesmo vocabulário. 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(dataset["Retificado"])
vocabulario=vectorizer.get_feature_names()  # usamos essa variável para obter o vocabulário de um vectorizer salvo via pickle



In [ ]:
#obtendo o y
y=dataset['Tag']

# Criando o modelo

Criaremos um com algoritmo Naive Bayes que é um algoritmo bem comum para análise de sentimento. Mas nada impede de utilizarmos outros que proporcionem um desempenho melhor.


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import metrics
from sklearn.metrics import confusion_matrix
nb = MultinomialNB()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=42,shuffle=True)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
nb.fit(X_train, y_train)
y_pred  = nb.predict(X_test)
print("Accuracy of model = %2f%%" % (accuracy_score(y_test, y_pred )*100))


Na própria base de dados, espere 70 - 80 % de precisão. 
Testando em outras bases de mensagens, o resultado fica por volta de 60%. 

# Testando em uma nova mensagem


In [ ]:
texto="Esse twitter é cheio de haters. Que droga!"
corpus=[texto]

#vetoriza
vectorizer = CountVectorizer(vocabulary=vocabulario)  # vamos vetorizar usando o vocabulário extraído da primeira vetorização
J  = vectorizer.fit_transform(corpus)


y_pred=nb.predict_proba(J)  #prevê probabilidade  %NEG  %POS [[0.8808889 0.1191111 ]]
#y_pred=nb.predict(J)       # retorna 0 (negativo)  ou 1 (positivo)
print(y_pred)